# Image Masking by U-NET Architecture From Scratch

![Masking Demo_image](https://analyticsindiamag.com/wp-content/uploads/2020/07/u-net-segmentation-e1542978983391.png)

**U-net Research Paper available at [unetpaper](https://arxiv.org/abs/1505.04597)**

**Blog on Image Segmentation and U-net. [check here](https://analyticsindiamag.com/my-experiment-with-unet-building-an-image-segmentation-model/)**

In [ ]:
# Importing Libraries

import os
import numpy as np
import pandas as pd
import tensorflow as tf
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
seed = 42
np.random.seed = seed
from sklearn.model_selection import train_test_split
from skimage.io import imread, imshow
from skimage.transform import resize

In [ ]:
# Defining Shape and Size of Image
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3

TRAIN_PATH = '/kaggle/input/clothing-coparsing-dataset'

In [ ]:
# Listing all image and mask name from file

X_imageid = os.listdir(os.path.join(TRAIN_PATH,'images'))
y_imageid = os.listdir(os.path.join(TRAIN_PATH,'labels','pixel_level_labels_colored'))
X_imageid.sort()
y_imageid.sort()

In [ ]:
# Creating zeros tensor of Input and output sizes. Later it will be filled with data.
X = np.zeros((len(X_imageid),IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS),dtype = np.uint8)
y = np.zeros((len(X_imageid),IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS),dtype = np.uint8)
print(X.shape,y.shape)

In [ ]:
# No. of Mask available as samplesize

samplesize = 1003
X_imageid = X_imageid[:samplesize]
y_imageid = y_imageid[:samplesize]
X = X[:samplesize]
y = y[:samplesize]

In [ ]:
# Loading data into zeros tensor variable as created above
for i,id_ in tqdm(enumerate(X_imageid),total = len(X_imageid)):
    path = os.path.join(TRAIN_PATH,'images',id_)
    img = imread(path)
    img = resize(img,(IMG_HEIGHT,IMG_WIDTH),mode='constant',preserve_range = True)
    X[i] =img
        
    mask_path = os.path.join(TRAIN_PATH,'labels','pixel_level_labels_colored',id_.split('.')[0]+'.png')

    img = imread(mask_path)
  
    img = resize(img,(IMG_HEIGHT,IMG_WIDTH),mode='constant',preserve_range = True)
    y[i] = img



In [ ]:
# Scaling by Normalizing data between 0 to 1
X = X/255.0
y = y/255.0

In [ ]:
# Spliting into train and valid data
X_train,X_valid,y_train,y_valid = train_test_split(X,y,random_state = seed,test_size = 0.2)
print(X_train.shape,X_valid.shape)

In [ ]:
# Random Sample View of image and image mask
image_x = random.randint(0, len(X_train)-1)
imshow(X_train[image_x])
plt.show()
imshow(np.squeeze(y_train[image_x]))
plt.show()

In [ ]:
# Building U-Net Architecture Model
def build_model():
    tf.keras.backend.clear_session()
    inputs = tf.keras.layers.Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))

    #Contraction path
    c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
    c1 = tf.keras.layers.Dropout(0.1)(c1)
    c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

    c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = tf.keras.layers.Dropout(0.1)(c2)
    c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

    c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = tf.keras.layers.Dropout(0.2)(c3)
    c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

    c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = tf.keras.layers.Dropout(0.2)(c4)
    c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)

    c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = tf.keras.layers.Dropout(0.3)(c5)
    c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

    #Expansive path 
    u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = tf.keras.layers.concatenate([u6, c4])
    c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = tf.keras.layers.Dropout(0.2)(c6)
    c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)

    u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = tf.keras.layers.concatenate([u7, c3])
    c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = tf.keras.layers.Dropout(0.2)(c7)
    c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)

    u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = tf.keras.layers.concatenate([u8, c2])
    c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = tf.keras.layers.Dropout(0.1)(c8)
    c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

    u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
    c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = tf.keras.layers.Dropout(0.1)(c9)
    c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

    outputs = tf.keras.layers.Conv2D(3, (1, 1), activation='sigmoid')(c9)

    model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:

model = build_model()
print("Input Shape :  ",model.input_shape)
print("Output Shape:  ",model.output_shape)
model.summary()

In [ ]:
checkpointer = tf.keras.callbacks.ModelCheckpoint('cloth_model.h5', verbose=1, save_best_only=True)
callbacks = [
        tf.keras.callbacks.TensorBoard(log_dir='logs')]

In [ ]:
# Training
results = model.fit(X_train, y_train, validation_split=0.1, batch_size=16, epochs=50, callbacks=callbacks)

In [ ]:
# Testing Data on Training Set
image_x = random.randint(0, len(X_train)-1)
imshow(X_train[image_x])
plt.show()
X_testing = np.expand_dims(X_train[image_x],axis = 0)

imshow(np.squeeze(y_train[image_x]))
plt.show()

prediction = model.predict(X_testing)

imshow(np.squeeze(prediction))
plt.show()

### Testing Data on Validation Set

In [ ]:
images_x = set()
while(len(images_x)!=10):
    images_x.add(random.randint(0, len(X_valid)-1))

for num,image_x in enumerate(images_x):
    plt.figure(figsize=(9,9))
    plt.subplot(1,3,1)
    plt.title("Real Image")
    imshow(X_valid[image_x])
    
    plt.subplot(1,3,2)
    plt.title("Labeled Output")
    imshow(np.squeeze(y_valid[image_x]))
    
    X_testing = np.expand_dims(X_valid[image_x],axis = 0)
    prediction = model.predict(X_testing)
    plt.subplot(1,3,3)
    plt.title('Predcted Output')
    imshow(np.squeeze(prediction))
    plt.show()

In [ ]:
model.save('model_unet.h5')

In [ ]:
def try_out_own_image(image_path):
    img = imread(image_path)
    img = resize(img,(IMG_HEIGHT,IMG_WIDTH),mode='constant',preserve_range = True)
    img = img/255.0
    print(img.shape)
    
    X_testing = np.expand_dims(img,axis = 0)
    prediction = model.predict(X_testing)
    plt.figure(figsize=(6,9))
    plt.subplot(1,2,1)
    plt.title('Real Image')
    imshow(img)
    plt.subplot(1,2,2)
    imshow(np.squeeze(prediction))
    plt.title('Predicted Output')
    plt.show()
    
    
    
    

In [ ]:
image_path = '../input/clothing-coparsing-dataset/images/1009.jpg'
try_out_own_image(image_path)

In [ ]:
image_path = '../input/clothing-coparsing-dataset/images/2067.jpg'
try_out_own_image(image_path)

In [ ]:
image_path = '../input/clothing-coparsing-dataset/images/1888.jpg'
try_out_own_image(image_path)